In [1]:
import numpy as np
import pandas as pd
import glob
import re

In [125]:
datadir = "../data/McGill-Billboard/"
# filepaths = glob.glob(datadir+'[0-9][0-9][0-9][0-9]/salami_chords.txt')
filepaths = glob.glob(datadir+'0010/salami_chords.txt')

In [126]:
fileidx = 0
filepath = filepaths[fileidx]

def ExtractFile(filepath):
    f = open(filepath, 'r')
    raw = f.readlines()
    f.close
    return raw

def ExtractMeta(raw):
    meta = ''.join(raw[:5])
    title = [x for x in re.split('# title: |\n', meta) if x and not x[0]=='#'][0]
    artist = [x for x in re.split('# artist: |\n', meta) if x and not x[0]=='#'][0]
    metre = [x for x in re.split('# metre: |\n', meta) if x and not x[0]=='#'][0]
    tonic = [x for x in re.split('# tonic: |\n', meta) if x and not x[0]=='#'][0]

    return (title, artist, metre, tonic)

raw = ExtractFile(filepath)
(title, artist, metre, tonic) = ExtractMeta(raw)

In [127]:
print((title, artist, metre, tonic))

('An Innocent Man', 'Billy Joel', '4/4', 'C')


In [139]:
def ExtractPhrases(raw):
    raw = raw[6:-2]
    phrases = list()
    for row in raw:
        splitted = re.split('\t', row)[1:]
        if not splitted: continue
        splitted = splitted[0]
        
        splitted = re.split('\|', splitted)[1:]        
        if not splitted: continue
        
        phrase = splitted[:-1]

        # check for repeat
        if IsRepeat(splitted[-1][1:3]):
            phrase.append(splitted[-1][1:3])

        phrases.append(phrase)
    return phrases

def CleanPhrases(phrases):
    phrases_clean = list()
    for phrase in phrases:
        phrase_clean = [chord.strip() for chord in phrase]
        phrases_clean.append(phrase_clean)
    return phrases_clean
    
def FormatChords(phrases_clean):
    chords = list()
    
    for phrase in phrases_clean:
        # clean non-chords
        phrase_chord = [RemoveNoise(chord) for chord in phrase]

        # remove blank chords
        phrase_chord = [p for p in phrase_chord if p!='']

        # remove blank (noise-only) lines
        if all([not p for p in phrase_chord]): continue

        # check for repeat
        if IsRepeat(phrase_chord[-1]):
            repeated = phrase_chord[:-1]
            n_repeat = int(phrase_chord[-1][-1])
            phrase_chord = repeated*n_repeat

        chords.append(phrase_chord)
    # flatten 2d list
    chords = sum(chords, [])

    # split chords into bars
    chords = [chord.split() for chord in chords]

    return chords

def IsRepeat(str):
    if re.match(r'x\d', str):
        return True
    else:
        return False

def RemoveNoise(chord):
    chord_clean = re.sub(r'|\*|\(\d+/\d+\)|N|&pause', '', chord)
    chord_clean = chord_clean.split()
    chord_clean = ' '.join(chord_clean)
    return chord_clean

phrases = ExtractPhrases(raw)
phrases_clean = CleanPhrases(phrases)
chords = FormatChords(phrases_clean)
# chords

phrase:  ['C:maj', 'x4']
phrase:  ['C:maj', 'C:maj', 'D:min', 'D:min', 'F:maj', 'G:maj', 'G:maj', 'C:maj']
phrase:  ['C:maj', 'C:maj', 'D:min', 'D:min', 'F:maj', 'G:maj', 'G:maj', 'C:maj']
phrase:  ['G:min7', 'C:maj/5', 'Bb:maj/5', 'F:maj', 'G:maj']
phrase:  ['C:maj', 'C:maj', 'D:min', 'D:min', 'F:maj', 'G:maj', 'G:maj', 'C:maj']
phrase:  ['C:maj', 'C:maj', 'D:min', 'D:min', 'F:maj', 'G:maj', 'G:maj']
phrase:  ['C:maj . F:maj/5 C:maj', 'C:maj F:maj/3 . G:maj/3']
phrase:  ['C:maj . F:maj/5 C:maj', 'C:maj D:min7 . C:maj/3']
phrase:  ['Bb:maj/5 F:maj', 'F:maj Bb:maj/5 . F:maj', 'Bb:maj/5 F:maj', 'F:maj G:maj . G:maj/3']
phrase:  ['C:maj', 'x4']
phrase:  ['C:maj', 'C:maj', 'D:min', 'D:min', 'F:maj', 'G:maj', 'G:maj', 'C:maj']
phrase:  ['C:maj', 'C:maj', 'D:min', 'D:min', 'F:maj', 'G:maj', 'G:maj', 'C:maj']
phrase:  ['G:min7', 'C:maj/5', 'Bb:maj/5', 'F:maj', 'G:maj']
phrase:  ['C:maj', 'C:maj', 'D:min', 'D:min', 'F:maj', 'G:maj', 'G:maj', 'C:maj']
phrase:  ['C:maj', 'C:maj', 'D:min', 'D:min

In [129]:
def CreateIndex(chords):
    indexs_bar = list()
    indexs_time= list()
    for i, bar in enumerate(chords):
        index_bar = [i]*len(bar)
        index_time= np.arange(i, i+1, 1/len(bar))
        indexs_bar.append(index_bar)
        indexs_time.append(list(index_time))
    return (indexs_bar, indexs_time)
(indexs_bar, indexs_time) = CreateIndex(chords)
chords

ZeroDivisionError: division by zero

In [130]:
def RemoveDots(chords, indexs_bar, indexs_time):
    for b_i, bar in enumerate(chords):
        for c_i, chord in enumerate(bar):
            if chord=='.':
                chords[b_i].pop(c_i)
                indexs_bar[b_i].pop(c_i)
                indexs_time[b_i].pop(c_i)
    return (chords, indexs_bar, indexs_time)

while '.' in sum(chords, []):
    (chords, indexs_bar, indexs_time) = RemoveDots(chords, indexs_bar, indexs_time)
chords

IndexError: pop index out of range

In [115]:
def Raw2Basic(chord_raw):
    root = chord_raw.split(':')[0]
    func = chord_raw.split(':')[1].split('/')[0]
    
    if func[:3]=='maj':
        func = 'maj'
    elif func[:3]=='min':
        func = 'min'
    elif func[:3]=='sus':
        func = func[:4]
    elif func[0]=='1' or func[0]=='5':
        func = '15'
    
    chord_basic = root+':'+func
    return chord_basic

def Raw2Root(chord_raw):
    root = chord_raw.split(':')[0]
    return root

In [116]:
def GetEnharmonic(note):
    if note == 'Db':
        return 'C#'
    elif note == 'Eb':
        return 'D#'
    elif note == 'E#':
        return 'F'
    elif note == 'Fb':
        return 'E'
    elif note == 'Gb':
        return 'F#'
    elif note == 'Ab':
        return 'G#'
    elif note == 'Bb':
        return 'A#'
    elif note == 'B#':
        return 'C'
    elif note == 'Cb':
        return 'B'
    return note

def Abs2RelRoot(chord_root, tonic):
    print(chord_root, tonic)
    chromatic = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
    relative = ['I', 'I#', 'II', 'II#', 'III', 'IV', 'IV#', 'V', 'V#', 'VI', 'VI#', 'VII']

    tonic = GetEnharmonic(tonic)
    chord_root = GetEnharmonic(chord_root)

    tonic_pos = chromatic.index(tonic)
    chord_root_pos = chromatic.index(chord_root)

    dist = chord_root_pos - tonic_pos
    rel_chord_root = relative[dist]

    return rel_chord_root

def Abs2Rel(chord, tonic):
    splitted = chord.split(':')
    root = splitted[0]
    func = ''
    if len(splitted) > 1:
        func = ':' + splitted[1]
    rel_root = Abs2RelRoot(root, tonic)
    rel_chord = rel_root + func

    return rel_chord

Abs2Rel('Bb:maj7', 'Ab')

Bb Ab


'II:maj7'

In [79]:
def Rel2InCRoot(root_rel, tonic='C'):
    chromatic = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
    relative = ['I', 'I#', 'II', 'II#', 'III', 'IV', 'IV#', 'V', 'V#', 'VI', 'VI#', 'VII']

    chromatic = chromatic[chromatic.index(tonic):] + chromatic[:chromatic.index(tonic)]
    root_inc = chromatic[relative.index(root_rel)]

    return root_inc

def Rel2InC(chord_rel, tonic='C'):
    splitted = chord_rel.split(':')
    root = splitted[0]
    func = ''
    if len(splitted) > 1:
        func = ':' + splitted[1]
    rel_root = Rel2InCRoot(root, tonic)
    rel_chord = rel_root + func

    return rel_chord

In [105]:
df_chords = pd.DataFrame()
df_chords['bar'] = sum(indexs_bar, [])
df_chords['bar_time'] = sum(indexs_time, [])
df_chords['chords'] = sum(chords, [])
df_chords['chords_basic'] = df_chords['chords'].apply(Raw2Basic)
df_chords['chords_root'] = df_chords['chords'].apply(Raw2Root)

df_chords['chords_rel'] = df_chords['chords'].apply(Abs2Rel, tonic=tonic)
df_chords['chords_basic_rel'] = df_chords['chords_basic'].apply(Abs2Rel, tonic=tonic)
df_chords['chords_root_rel'] = df_chords['chords_root'].apply(Abs2Rel, tonic=tonic)

df_chords['chords_inC'] = df_chords['chords_rel'].apply(Rel2InC)
df_chords['chords_basic_inC'] = df_chords['chords_basic_rel'].apply(Rel2InC)
df_chords['chords_root_inC'] = df_chords['chords_root_rel'].apply(Rel2InC)

df_chords['meta'] = ['title: '+title, 'artist: '+artist, 'metre: '+metre, 'tonic: '+tonic] + [np.nan] * (len(df_chords)-4)

E E
E E
E E
E E
E E
G E
A E
E E
E E
E E
E E
G E
A E
E E
E E
E E
E E
G E
A E
E E
E E
E E
E E
G E
A E
E E
E E
E E
E E
G E
A E
E E
E E
E E
E E
G E
A E
E E
E E
E E
E E
G E
A E
E E
E E
E E
E E
G E
A E
E E
E E
E E
E E
G E
A E
E E
E E
E E
E E
G E
A E
E E
E E
E E
E E
G E
A E
E E
E E
E E
E E
G E
A E
E E
E E
E E
E E
G E
A E
E E
E E
E E
E E
G E
A E
E E
E E
E E
E E
G E
A E
E E
E E
E E
E E
G E
A E
E E
E E
E E
E E
G E
A E
E E
E E
E E
E E
G E
A E
E E
E E
E E
E E
G E
A E
E E
E E
E E
E E
G E
A E
E E
E E
E E
E E
G E
A E
E E
E E
E E
E E
G E
A E
E E
E E
E E
E E
E E
E E
E E
E E
G E
A E
E E
E E
E E
E E
G E
A E
E E
E E
E E
E E
G E
A E
E E
E E
E E
E E
G E
A E
E E
E E
E E
E E
G E
A E
E E
E E
E E
E E
G E
A E
E E
E E
E E
E E
G E
A E
E E
E E
E E
E E
G E
A E
E E
E E
E E
E E
G E
A E
E E
E E
E E
E E
G E
A E
E E
E E
E E
E E
G E
A E
E E
E E
E E
E E
G E
A E
E E
E E
E E
E E
G E
A E
E E
E E
E E
E E
G E
A E
E E
E E
E E
E E
G E
A E
E E
E E
E E
E E
G E
A E
E E
E E
E E
E E
G E
A E
E E
E E
E E
E E
G E
A E
E E
E E
E E
E E
G E


In [106]:
np.array(df_chords['chords'])

array(['E:1', 'E:maj(11)', 'E:7(#9)', 'E:7(#9)', 'E:7(#9)', 'G:1', 'A:1',
       'E:7(#9)', 'E:7(#9)', 'E:7(#9)', 'E:7(#9)', 'G:1', 'A:1',
       'E:7(#9)', 'E:7(#9)', 'E:7(#9)', 'E:7(#9)', 'G:1', 'A:1',
       'E:7(#9)', 'E:7(#9)', 'E:7(#9)', 'E:maj(#9)', 'G:maj', 'A:maj',
       'E:7(#9)', 'E:7(#9)', 'E:7(#9)', 'E:maj(#9)', 'G:maj', 'A:maj',
       'E:7(#9)', 'E:7(#9)', 'E:7(#9)', 'E:maj(#9)', 'G:maj', 'A:maj',
       'E:7(#9)', 'E:7(#9)', 'E:7(#9)', 'E:maj(#9)', 'G:maj', 'A:maj',
       'E:7(#9)', 'E:7(#9)', 'E:7(#9)', 'E:maj(#9)', 'G:maj', 'A:maj',
       'E:7(#9)', 'E:7(#9)', 'E:7(#9)', 'E:maj(#9)', 'G:maj', 'A:maj',
       'E:7(#9)', 'E:7(#9)', 'E:7(#9)', 'E:maj(#9)', 'G:maj', 'A:maj',
       'E:7(#9)', 'E:7(#9)', 'E:7(#9)', 'E:maj(#9)', 'G:maj', 'A:maj',
       'E:7(#9)', 'E:7(#9)', 'E:7(#9)', 'E:maj(#9)', 'G:maj', 'A:maj',
       'E:7(#9)', 'E:7(#9)', 'E:7(#9)', 'E:maj(#9)', 'G:maj', 'A:maj',
       'E:7(#9)', 'E:7(#9)', 'E:7(#9)', 'E:maj(#9)', 'G:maj', 'A:maj',
       'E:7(#9)

In [107]:
csvpath = '/'.join(filepath.split('/')[:-1]) + '/chords.csv'
df_chords.to_csv(csvpath)

In [108]:
df_chords

,bar,bar_time,chords,chords_basic,chords_root,chords_rel,chords_basic_rel,chords_root_rel,chords_inC,chords_basic_inC,chords_root_inC,meta
0,0,0.00,E:1,E:15,E,I:1,I:15,I,C:1,C:15,C,title: On The Road Again
1,1,1.00,E:maj(11),E:maj,E,I:maj(11),I:maj,I,C:maj(11),C:maj,C,artist: Canned Heat
2,2,2.00,E:7(#9),E:7(#9),E,I:7(#9),I:7(#9),I,C:7(#9),C:7(#9),C,metre: 4/4
3,3,3.00,E:7(#9),E:7(#9),E,I:7(#9),I:7(#9),I,C:7(#9),C:7(#9),C,tonic: E
4,4,4.00,E:7(#9),E:7(#9),E,I:7(#9),I:7(#9),I,C:7(#9),C:7(#9),C,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
131,88,88.50,G:maj,G:maj,G,II#:maj,II#:maj,II#,D#:maj,D#:maj,D#,NaN
132,88,88.75,A:maj,A:maj,A,IV:maj,IV:maj,IV,F:maj,F:maj,F,NaN
133,89,89.00,E:7(#9),E:7(#9),E,I:7(#9),I:7(#9),I,C:7(#9),C:7(#9),C,NaN
134,90,90.00,E:1,E:15,E,I:1,I:15,I,C:1,C:15,C,NaN
